In [1]:
import numpy as np
import requests
import json
import pandas as pd
import time
from webbrowser import get

# https://developer.riotgames.com/docs/lol -- intro
# https://developer.riotgames.com/apis#league-v4/GET_getLeagueEntriesForSummoner -- docs
# https://riot-api-libraries.readthedocs.io/en/latest/libraries.html#python -- helpers/packages that use API

In [9]:
# Get API key
with open('../../API_KEY.txt') as f:
    apiKey = f.readline()

# Match IDs

In [3]:
# Get last N soloq ranked games of a player (puuid)
puuid = 'U1I7G1e8NOl94cJ1Etg42k3oVvq5R9ns8S89xzu_MAcVkhmOfXMb2VkEIloAZIiyggc3N0AIYDzaIg'
lastNGames = 20
urlMatchIds = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count={lastNGames}'
matches = requests.get(urlMatchIds, headers={"X-Riot-Token": apiKey})

# Save to dict
matchIds = {'Match ID': matches.json(), 'Player': [puuid for _ in matches.json()]}

In [15]:
# Load database of match IDs
matchIdsDatabase = pd.read_csv('matchIds.csv',  index_col=[0])

# Check if match is in the database, if not add it
#[matchIdsDatabase.append({'Match ID': match, 'Player': puuid}) for match in matchIds['Match ID'] if match not in matchIdsDatabase.values]

# Retrieve following information from each match: players, champions, and masteries

In [123]:
# Get information on match 0
urlMatchInfo = f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchIdsDatabase["Match ID"][0]}'
matchInfo = pd.Series(requests.get(urlMatchInfo, headers={"X-Riot-Token": apiKey}).json())

In [124]:
# Gather PUUIDs of players in the match
teamOnePUUID, teamTwoPUUID = [], []
for i in range(5):
    teamOnePUUID.append(matchInfo[1]['participants'][i]['puuid'])
    teamTwoPUUID.append(matchInfo[1]['participants'][i+5]['puuid'])

In [95]:
# Gather summoner ids of players in the match
teamOneSummId, teamTwoSummId = [], []
for i in range(5):
    teamOneSummId.append(matchInfo[1]['participants'][i]['summonerId'])
    teamTwoSummId.append(matchInfo[1]['participants'][i+5]['summonerId'])

In [73]:
# Gather champions played in the match
teamOneChamps, teamTwoChamps = [], []
for i in range(5):
    teamOneChamps.append(matchInfo[1]['participants'][i]['championId'])
    teamTwoChamps.append(matchInfo[1]['participants'][i+5]['championId'])

In [104]:
# Gather mastery scores for each player/champion in the match
teamOneMasteries, teamTwoMasteries = [], []

for i, champion in enumerate(teamOneChamps):
    urlChampionMastery = f'https://euw1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{teamOneSummId[i]}/by-champion/{champion}'
    masteryPoints = pd.Series(requests.get(urlChampionMastery, headers={"X-Riot-Token": apiKey}).json())
    teamOneMasteries.append(masteryPoints['championPoints'])

for i, champion in enumerate(teamTwoChamps):
    urlChampionMastery = f'https://euw1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{teamTwoSummId[i]}/by-champion/{champion}'
    masteryPoints = pd.Series(requests.get(urlChampionMastery, headers={"X-Riot-Token": apiKey}).json())
    teamTwoMasteries.append(masteryPoints['championPoints'])

In [110]:
# Collect into dict
data = {'Match ID': 'match1',
        'Team One PUUID': teamOnePUUID, 'Team Two PUUID': teamTwoPUUID,
        'Team One Champions': teamOneChamps, 'Team Two Champions': teamTwoChamps,
        'Team One Masteries': teamOneMasteries, 'Team Two Masteries': teamTwoMasteries}

In [126]:
# Dict w/ champion ids
with open('champion_ids.txt') as champ_ids:
    champs = {}
    for i, line in enumerate(champ_ids):
        for j, char in enumerate(line):
            if char == ':':
                champs[f'{line[j+1:]}'.strip()] = int(line[:j])